In [1]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "10g") 
    .getOrCreate()
)

22/09/01 13:47:25 WARN Utils: Your hostname, DESKTOP-5U9FK5P resolves to a loopback address: 127.0.1.1; using 172.23.211.30 instead (on interface eth0)
22/09/01 13:47:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/01 13:47:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [27]:
from pyspark.sql.functions import *

In [26]:
# Reading in cleaned data
sdf_merchants = spark.read.parquet('../data/tables/tbl_merchants.parquet')
sdf_merchants.show(2, truncate=False)

+------------------------+-----------------------------------------------------------------------------------------------------------------+------------+
|name                    |tags                                                                                                             |merchant_abn|
+------------------------+-----------------------------------------------------------------------------------------------------------------+------------+
|Felis Limited           |((furniture, home furnishings and equipment shops, and manufacturers, except appliances), (e), (take rate: 0.18))|10023283211 |
|Arcu Ac Orci Corporation|([cable, satellite, and otHer pay television and radio services], [b], [take rate: 4.22])                        |10142254217 |
+------------------------+-----------------------------------------------------------------------------------------------------------------+------------+
only showing top 2 rows



In [14]:
sdf_merchants.printSchema()

root
 |-- name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- merchant_abn: long (nullable = true)



In [12]:
sdf_merchants.count()

4026

In [37]:
sdf_consumer = spark.read.csv('../data/tables/tbl_consumer.csv', sep='|', header=True)
sdf_consumer.show(5, truncate=False)

+-----------------+-----------------------------+-----+--------+------+-----------+
|name             |address                      |state|postcode|gender|consumer_id|
+-----------------+-----------------------------+-----+--------+------+-----------+
|Yolanda Williams |413 Haney Gardens Apt. 742   |WA   |6935    |Female|1195503    |
|Mary Smith       |3764 Amber Oval              |NSW  |2782    |Female|179208     |
|Jill Jones MD    |40693 Henry Greens           |NT   |862     |Female|1194530    |
|Lindsay Jimenez  |00653 Davenport Crossroad    |NSW  |2780    |Female|154128     |
|Rebecca Blanchard|9271 Michael Manors Suite 651|WA   |6355    |Female|712975     |
+-----------------+-----------------------------+-----+--------+------+-----------+
only showing top 5 rows



In [13]:
sdf_consumer.count()

500000

In [15]:
sdf_consumer.printSchema()

root
 |-- _c0: string (nullable = true)



In [20]:
sdf_transactions = spark.read.parquet('../data/tables/transactions_20210228_20210827_snapshot')
sdf_transactions.show(10, truncate=False)

+-------+------------+------------------+------------------------------------+--------------+
|user_id|merchant_abn|dollar_value      |order_id                            |order_datetime|
+-------+------------+------------------+------------------------------------+--------------+
|18478  |62191208634 |63.255848959735246|949a63c8-29f7-4ab0-ada4-99ac50a88952|2021-08-20    |
|2      |15549624934 |130.3505283105634 |6a84c3cf-612a-4574-835b-144a47353eff|2021-08-20    |
|18479  |64403598239 |120.15860593212783|b10dcc33-e53f-4254-863c-de5266810cbc|2021-08-20    |
|3      |60956456424 |136.6785200286976 |0f09c5a5-784e-4477-b049-8ee4dd069b7b|2021-08-20    |
|18479  |94493496784 |72.96316578355305 |f6c78c1a-4600-4c5f-8e97-6e9eb534b586|2021-08-20    |
|3      |76819856970 |448.529684285612  |5ace6a24-cdf0-4aa3-b571-1d9406b352b5|2021-08-20    |
|18479  |67609108741 |86.4040605836911  |d0e180f0-cb06-42a3-bd1a-c47dca15bc55|2021-08-20    |
|3      |34096466752 |301.5793450525113 |6fb1ff48-24bb-4f97-

In [18]:
sdf_transactions.count()

3643266

In [19]:
sdf_transactions.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



In [25]:
sdf_userdetails = spark.read.parquet('../data/tables/consumer_user_details.parquet')
sdf_userdetails.show(10, truncate=False)

+-------+-----------+
|user_id|consumer_id|
+-------+-----------+
|1      |1195503    |
|2      |179208     |
|3      |1194530    |
|4      |154128     |
|5      |712975     |
|6      |407340     |
|7      |511685     |
|8      |448088     |
|9      |650435     |
|10     |1058499    |
+-------+-----------+
only showing top 10 rows



In [40]:
sdf_usertransaction = sdf_userdetails.join(sdf_transactions, on='user_id')
sdf_consumer_transaction = sdf_usertransaction.join(sdf_consumer, on='consumer_id')
sdf_consumer_transaction.show(5)

+-----------+-------+------------+------------------+--------------------+--------------+---------+--------------------+-----+--------+------+
|consumer_id|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|     name|             address|state|postcode|gender|
+-----------+-------+------------+------------------+--------------------+--------------+---------+--------------------+-----+--------+------+
|        551|   3471| 68216911708|  39.2325218595659|c921263e-489e-45a...|    2021-08-21|Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|
|        551|   3471| 77698107389| 90.16967584866273|aaea14f5-48e8-4ef...|    2021-08-21|Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|
|        551|   3471| 46451548968|3.8405569898888428|7e8e7fca-04a1-4f2...|    2021-08-19|Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|
|        551|   3471| 35556933338| 66.18325079406301|13b0be6a-20b7-439...|    2021-08-22|Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|

In [41]:
sdf_consumer_transaction = sdf_consumer_transaction.select("consumer_id","user_id",'merchant_abn','dollar_value',
                            'order_datetime','state','postcode','gender')
sdf_consumer_transaction.show(5)                   

+-----------+-------+------------+------------------+--------------+-----+--------+------+
|consumer_id|user_id|merchant_abn|      dollar_value|order_datetime|state|postcode|gender|
+-----------+-------+------------+------------------+--------------+-----+--------+------+
|        551|   3471| 68216911708|  39.2325218595659|    2021-08-21|  NSW|    2021|  Male|
|        551|   3471| 77698107389| 90.16967584866273|    2021-08-21|  NSW|    2021|  Male|
|        551|   3471| 46451548968|3.8405569898888428|    2021-08-19|  NSW|    2021|  Male|
|        551|   3471| 35556933338| 66.18325079406301|    2021-08-22|  NSW|    2021|  Male|
|        551|   3471| 91772719127|33.692747732339996|    2021-08-14|  NSW|    2021|  Male|
+-----------+-------+------------+------------------+--------------+-----+--------+------+
only showing top 5 rows

